#Обучение mt5

In [ ]:
!pip freeze > requirements.txt

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip uninstall -y transformers tokenizers accelerate datasets huggingface_hub safetensors
!pip install -U --no-cache-dir \
  "transformers==4.57.3" \
  "accelerate" \
  "datasets" \
  "sentencepiece" \
  "safetensors"


In [ ]:
import transformers, accelerate
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)


transformers: 4.57.3
accelerate: 1.12.0


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import os
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)

MODEL_NAME = "google/mt5-small"
OUTPUT_DIR = "mt5_fact_extractor"
MAX_SOURCE_LEN = 256
MAX_TARGET_LEN = 256

set_seed(42)



In [ ]:
raw_dataset = load_dataset("json", data_files="/content/dataset.jsonl")
raw_dataset = raw_dataset.shuffle()

In [ ]:
ds = raw_dataset["train"].train_test_split(test_size=0.1)
ds["validation"] = ds.pop("test")

In [ ]:
def join_facts(facts):
    # Канонизация "множества" в последовательность:
    # uniq + sort, чтобы снизить шум порядка в Seq2Set постановке
    facts = [f.strip() for f in facts if f and f.strip()]
    facts = list(dict.fromkeys(facts))  # uniq (сохр. порядок)
    facts = sorted(facts)               # канон. порядок
    # Вставляем теги
    facts = [f"<FACT>{f}</FACT>" for f in facts]
    return "\n".join(facts)

In [ ]:
from transformers import AutoTokenizer

checkpoint = MODEL_NAME
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def preprocess_function(examples, tokenizer):
    inputs = examples["text"]
    targets = join_facts(examples["facts"])
    model_inputs = tokenizer(inputs, text_target=targets, max_length=256, truncation=True)
    return model_inputs

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Добавим спец-токены, чтобы <FACT> не распадался
special_tokens = {"additional_special_tokens": ["<FACT>", "</FACT>"]}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Embedding(250102, 512)

In [ ]:
tokenized = ds.map(
    lambda ex: preprocess_function(ex, tokenizer),
    remove_columns=ds["train"].column_names,
    desc="Tokenizing",
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Tokenizing:   0%|          | 0/216 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/24 [00:00<?, ? examples/s]

In [ ]:
def extract_facts_from_generated(s: str):
    out = []
    i = 0
    while True:
        a = s.find("<FACT>", i)
        if a == -1:
            break
        b = s.find("</FACT>", a)
        if b == -1:
            break
        fact = s[a+len("<FACT>"):b].strip()
        if fact:
            out.append(fact)
        i = b + len("</FACT>")
    return list(dict.fromkeys(out))

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=False)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=False)

    f1s = []
    nonempty = []
    for p, y in zip(pred_texts, label_texts):
        pf = set(extract_facts_from_generated(p))
        yf = set(extract_facts_from_generated(y))
        nonempty.append(1.0 if len(pf) > 0 else 0.0)

        tp = len(pf & yf)
        prec = tp / max(len(pf), 1)
        rec = tp / max(len(yf), 1)
        f1 = 0.0 if (prec + rec) == 0 else (2 * prec * rec / (prec + rec))
        f1s.append(f1)

    return {
        "format_nonempty_rate": float(np.mean(nonempty)),
        "fact_set_f1": float(np.mean(f1s)),
    }

In [ ]:
import torch

# Очистка памяти перед стартом
torch.cuda.empty_cache()

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2, # Можно увеличить накопление градиента для стабильности
    learning_rate=3e-4,            # Чуть меньше, чем 5e-4
    warmup_ratio=0.05,
    num_train_epochs=10,

    # ВАЖНО ДЛЯ T4 и mT5:
    fp16=False,                    # Выключаем Mixed Precision, чтобы избежать NaN
    optim="adafactor",             # Родной оптимизатор для T5
    gradient_checkpointing=False,  # Иногда помогает выключение, если памяти хватает

    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LEN,
    generation_num_beams=4,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer, # В новых версиях transformers лучше использовать processing_class=tokenizer
    #compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-1184673264.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 000cabbf-c1c8-4af7-a6d2-47814b3cecba)')' thrown while requesting HEAD https://huggingface.co/google/mt5-small/resolve/refs%2Fpr%2F15/model.safetensors
Retrying in 1s [Retry 1/5].


model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,21.903600,14.696309
2,12.290800,4.531689
3,5.907900,3.706713
4,4.815000,2.737902
5,3.687500,2.529600
6,3.516300,2.305413
7,3.199900,2.196126
8,2.997600,2.139407
9,2.965800,2.087487
10,2.837800,2.066621


TrainOutput(global_step=140, training_loss=5.847659369877406, metrics={'train_runtime': 444.9987, 'train_samples_per_second': 4.854, 'train_steps_per_second': 0.315, 'total_flos': 544992531578880.0, 'train_loss': 5.847659369877406, 'epoch': 10.0})

In [ ]:
trainer.save_model(os.path.join(OUTPUT_DIR, "final"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "final"))

print("Saved to:", os.path.join(OUTPUT_DIR, "final"))

Saved to: mt5_fact_extractor/final


какой-то инференс

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tok = AutoTokenizer.from_pretrained("mt5_fact_extractor/final")
mdl = AutoModelForSeq2SeqLM.from_pretrained("mt5_fact_extractor/final").eval()

def gen_facts(text: str):
    inp = tok(text, return_tensors="pt", truncation=True, max_length=256)
    out = mdl.generate(
        **inp,
        max_length=256,
        num_beams=4,
        do_sample=False,           # детерминированно
        repetition_penalty=1.1,
    )
    decoded = tok.decode(out[0], skip_special_tokens=False)

    # Парсим <FACT>...</FACT>
    facts = []
    i = 0
    while True:
        a = decoded.find("<FACT>", i)
        if a == -1:
            break
        b = decoded.find("</FACT>", a)
        if b == -1:
            break
        fact = decoded[a+len("<FACT>"):b].strip()
        if fact:
            facts.append(fact)
        i = b + len("</FACT>")

    # Строгая экстрактивность: оставляем только точные подстроки
    facts = list(dict.fromkeys(facts))
    facts_extractive = [f for f in facts if f in text]
    return decoded, facts_extractive

test_text = "Компания «Ромашка» основана в 1999 году в Москве. Музей работает с 10:00 до 18:00 ежедневно."
raw, facts = gen_facts(test_text)

print("=== RAW ===")
print(raw)
print("\n=== EXTRACTIVE FACTS (fact in text) ===")
for f in facts:
    print("-", f)


The tokenizer you are loading from 'mt5_fact_extractor/final' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


=== RAW ===
<pad> <extra_id_0> Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей «Ромашка» основана в 1999 году Музей

=== EXTRACTIVE FACTS (fact in text) ===
